In [ ]:
# default_exp blocker
%load_ext autoreload
%autoreload 2

# Website Blocker

> This is the core functionality of beproductive. A defined list of websites is added to the `hosts` file of the OS with a redirect to `127.0.0.1`.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from pathlib import Path
from shutil import copy
import sys
try:
    from win10toast import ToastNotifier
    win_notify = ToastNotifier()
except:
    win_notify = False

In [ ]:
#export
APP_NAME = 'Website Blocker'
REDIRECT = '127.0.0.1'
WIN_PATH = r'C:\Windows\System32\drivers\etc'
ICON_PATH = 'icon.ico'
BLOCKLIST = 'blocklist.txt'

host_path = Path(WIN_PATH)
host_file = host_path/'hosts'
host_file_copy = host_path/'hosts.original'
host_file_blocked = host_path/'hosts.blocked'

In [ ]:
#export
class Blocker():
    "The core of the package. It modifies the hosts file of the OS."
    def __init__(self, redirect=REDIRECT):
        self.redirect = redirect
        with open(BLOCKLIST) as file:
            self.blocked = [line.rstrip() for line in file]
        if not host_file_copy.exists():
            self._setup()
        self._create_blocked_list()

    def _setup(self):
        "Creates a copy of the `hosts` file and saves it as `hosts.original`"
        try:
            copy(host_file, host_file_copy)
            print("Copy of hosts file created")
        except PermissionError:
            self._raise_permission_error()

    def _create_blocked_list(self):
        "Creates a copy of `hosts.original` and saves it to `hosts.blocked`. Then adds all blocked sites."
        try:
            copy(host_file_copy, host_file_blocked)
            with open(host_file_blocked, "a") as blocked_file:
                for url in self.blocked:
                    # TODO: refine, add www only if not in url
                    blocked_file.write(f"{self.redirect} {url}\n")
                    blocked_file.write(f"{self.redirect} www.{url}\n")
                print("hosts.blocked created")
        except PermissionError:
            self._raise_permission_error()

    def block(self):
        "Blocks all specified websites by replacing `hosts` file with `hosts.blocked`"
        try:
            copy(host_file_blocked, host_file)
            print("Hosts file replaced with hosts.blocked")
            return True
        except PermissionError:
            self._raise_permission_error()
            return False

    def unblock(self):
        "Unblocks all websites by restoring the original `hosts` file"
        try:
            copy(host_file_copy, host_file)
            print("Hosts file replaced with hosts.original")
            return True
        except PermissionError:
            self._raise_permission_error()
            return False

    def notify(self, message, title=APP_NAME):
        "Sends notification to CLI and - if available - to GUI"
        print(message)
        if win_notify:
            win_notify.show_toast(title, message, duration=5) # icon_path=ICON_PATH, 

    def _raise_permission_error(self):
        self.notify("Permission Error. Please run the command line tool as ADMINISTRATOR.")

In [ ]:
show_doc(Blocker.block)

<h4 id="Blocker.block" class="doc_header"><code>Blocker.block</code><a href="__main__.py#L33" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.block</code>()

Blocks all specified websites by replacing `hosts` file with `hosts.blocked`

In [ ]:
show_doc(Blocker.unblock)

<h4 id="Blocker.unblock" class="doc_header"><code>Blocker.unblock</code><a href="__main__.py#L43" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.unblock</code>()

Unblocks all websites by restoring the original `hosts` file

In [ ]:
show_doc(Blocker.notify)

<h4 id="Blocker.notify" class="doc_header"><code>Blocker.notify</code><a href="__main__.py#L53" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.notify</code>(**`message`**, **`title`**=*`'Website Blocker'`*)

Sends notification to CLI and - if available - to GUI

## Using Blocker

In [ ]:
blocker = Blocker()
blocker.block()

hosts.blocked created
Hosts file replaced with hosts.blocked


True

In [ ]:
blocker.unblock()

Hosts file replaced with hosts.original


True

## Command line tool

In [ ]:
#export
if __name__=='__main__':
    blocker = Blocker()
    try: mode = sys.argv[1].lower()
    except: mode = 'none' 

    if mode == 'block':
        if blocker.block():
            blocker.notify("Websites blocked, enjoy your work")
    elif mode == 'unblock':
        if blocker.unblock():
            blocker.notify("Websites unblocked, have fun")
    else:
        print("Run blocker.py with arguments BLOCK or UNBLOCK")

hosts.blocked created
Run blocker.py with arguments BLOCK or UNBLOCK
